# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

# wk_debug
from torch.optim import lr_scheduler
import nltk
nltk.download('punkt')

## TODO #1: Select appropriate values for the Python variables below.
batch_size = 256          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
#optimizer = torch.optim.SGD(params, lr=0.001)
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)
optimizer = torch.optim.Adam(params)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.94s)
creating index...


  0%|          | 494/414113 [00:00<01:23, 4938.00it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:04<00:00, 6397.79it/s]


__init__(embed_size, hidden_size, vocab_size, num_layers): 512 512 8855 1


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [2]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        # wk_debug
        #exp_lr_scheduler.step()
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # wk_debug
        #print('images:', images.size())
        #print('captions:', captions.size())
        #print('captions[0]:', captions[0])
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-0621-epoch%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-0621-epoch%d.pkl' % epoch))

# Close the training log file.
f.close()

lstm_out: torch.Size([256, 11, 512])
tag_space: torch.Size([256, 11, 8855])
Epoch [1/3], Step [1/1618], Loss: 9.0942, Perplexity: 8903.8092lstm_out: torch.Size([256, 13, 512])
tag_space: torch.Size([256, 13, 8855])
Epoch [1/3], Step [2/1618], Loss: 8.8378, Perplexity: 6889.6740lstm_out: torch.Size([256, 11, 512])
tag_space: torch.Size([256, 11, 8855])
Epoch [1/3], Step [3/1618], Loss: 8.5382, Perplexity: 5105.9765lstm_out: torch.Size([256, 10, 512])
tag_space: torch.Size([256, 10, 8855])
Epoch [1/3], Step [4/1618], Loss: 8.1576, Perplexity: 3489.8690lstm_out: torch.Size([256, 13, 512])
tag_space: torch.Size([256, 13, 8855])
Epoch [1/3], Step [5/1618], Loss: 7.6881, Perplexity: 2182.2449lstm_out: torch.Size([256, 14, 512])
tag_space: torch.Size([256, 14, 8855])
Epoch [1/3], Step [6/1618], Loss: 7.1922, Perplexity: 1329.0849lstm_out: torch.Size([256, 12, 512])
tag_space: torch.Size([256, 12, 8855])
Epoch [1/3], Step [7/1618], Loss: 6.4137, Perplexity: 610.1602lstm_out: torch.Size([256, 1

Epoch [1/3], Step [60/1618], Loss: 3.8408, Perplexity: 46.5613lstm_out: torch.Size([256, 12, 512])
tag_space: torch.Size([256, 12, 8855])
Epoch [1/3], Step [61/1618], Loss: 3.8759, Perplexity: 48.2260lstm_out: torch.Size([256, 12, 512])
tag_space: torch.Size([256, 12, 8855])
Epoch [1/3], Step [62/1618], Loss: 3.8458, Perplexity: 46.7944lstm_out: torch.Size([256, 15, 512])
tag_space: torch.Size([256, 15, 8855])
Epoch [1/3], Step [63/1618], Loss: 3.8453, Perplexity: 46.7738lstm_out: torch.Size([256, 13, 512])
tag_space: torch.Size([256, 13, 8855])
Epoch [1/3], Step [64/1618], Loss: 3.6554, Perplexity: 38.6839lstm_out: torch.Size([256, 18, 512])
tag_space: torch.Size([256, 18, 8855])
Epoch [1/3], Step [65/1618], Loss: 4.3001, Perplexity: 73.7072lstm_out: torch.Size([256, 13, 512])
tag_space: torch.Size([256, 13, 8855])
Epoch [1/3], Step [66/1618], Loss: 3.7505, Perplexity: 42.5409lstm_out: torch.Size([256, 12, 512])
tag_space: torch.Size([256, 12, 8855])
Epoch [1/3], Step [67/1618], Loss:

Epoch [1/3], Step [119/1618], Loss: 3.6117, Perplexity: 37.0273lstm_out: torch.Size([256, 14, 512])
tag_space: torch.Size([256, 14, 8855])
Epoch [1/3], Step [120/1618], Loss: 3.4062, Perplexity: 30.1490lstm_out: torch.Size([256, 12, 512])
tag_space: torch.Size([256, 12, 8855])
Epoch [1/3], Step [121/1618], Loss: 3.4805, Perplexity: 32.4761lstm_out: torch.Size([256, 11, 512])
tag_space: torch.Size([256, 11, 8855])
Epoch [1/3], Step [122/1618], Loss: 3.5654, Perplexity: 35.3546lstm_out: torch.Size([256, 11, 512])
tag_space: torch.Size([256, 11, 8855])
Epoch [1/3], Step [123/1618], Loss: 3.5311, Perplexity: 34.1601lstm_out: torch.Size([256, 15, 512])
tag_space: torch.Size([256, 15, 8855])
Epoch [1/3], Step [124/1618], Loss: 3.4574, Perplexity: 31.7331lstm_out: torch.Size([256, 23, 512])
tag_space: torch.Size([256, 23, 8855])
Epoch [1/3], Step [125/1618], Loss: 4.3108, Perplexity: 74.5011lstm_out: torch.Size([256, 14, 512])
tag_space: torch.Size([256, 14, 8855])
Epoch [1/3], Step [126/1618

Epoch [1/3], Step [178/1618], Loss: 3.1508, Perplexity: 23.3542lstm_out: torch.Size([256, 13, 512])
tag_space: torch.Size([256, 13, 8855])
Epoch [1/3], Step [179/1618], Loss: 3.1946, Perplexity: 24.4007lstm_out: torch.Size([256, 13, 512])
tag_space: torch.Size([256, 13, 8855])
Epoch [1/3], Step [180/1618], Loss: 3.1874, Perplexity: 24.2244lstm_out: torch.Size([256, 13, 512])
tag_space: torch.Size([256, 13, 8855])
Epoch [1/3], Step [181/1618], Loss: 3.1342, Perplexity: 22.9693lstm_out: torch.Size([256, 14, 512])
tag_space: torch.Size([256, 14, 8855])
Epoch [1/3], Step [182/1618], Loss: 3.1686, Perplexity: 23.7749lstm_out: torch.Size([256, 18, 512])
tag_space: torch.Size([256, 18, 8855])
Epoch [1/3], Step [183/1618], Loss: 3.6913, Perplexity: 40.0978lstm_out: torch.Size([256, 11, 512])
tag_space: torch.Size([256, 11, 8855])
Epoch [1/3], Step [184/1618], Loss: 3.2498, Perplexity: 25.7846lstm_out: torch.Size([256, 14, 512])
tag_space: torch.Size([256, 14, 8855])
Epoch [1/3], Step [185/1618

Epoch [1/3], Step [237/1618], Loss: 3.1322, Perplexity: 22.9251lstm_out: torch.Size([256, 15, 512])
tag_space: torch.Size([256, 15, 8855])
Epoch [1/3], Step [238/1618], Loss: 3.1304, Perplexity: 22.8841lstm_out: torch.Size([256, 16, 512])
tag_space: torch.Size([256, 16, 8855])
Epoch [1/3], Step [239/1618], Loss: 3.2404, Perplexity: 25.5449lstm_out: torch.Size([256, 11, 512])
tag_space: torch.Size([256, 11, 8855])
Epoch [1/3], Step [240/1618], Loss: 3.1536, Perplexity: 23.4208lstm_out: torch.Size([256, 12, 512])
tag_space: torch.Size([256, 12, 8855])
Epoch [1/3], Step [241/1618], Loss: 3.0971, Perplexity: 22.1345lstm_out: torch.Size([256, 13, 512])
tag_space: torch.Size([256, 13, 8855])
Epoch [1/3], Step [242/1618], Loss: 3.0659, Perplexity: 21.4532lstm_out: torch.Size([256, 12, 512])
tag_space: torch.Size([256, 12, 8855])
Epoch [1/3], Step [243/1618], Loss: 3.1477, Perplexity: 23.2821lstm_out: torch.Size([256, 14, 512])
tag_space: torch.Size([256, 14, 8855])
Epoch [1/3], Step [244/1618

...

Epoch [3/3], Step [131/1618], Loss: 2.0388, Perplexity: 7.6816lstm_out: torch.Size([256, 18, 512])
tag_space: torch.Size([256, 18, 8855])
Epoch [3/3], Step [132/1618], Loss: 2.4671, Perplexity: 11.7887lstm_out: torch.Size([256, 13, 512])
tag_space: torch.Size([256, 13, 8855])
Epoch [3/3], Step [133/1618], Loss: 1.9064, Perplexity: 6.7290lstm_out: torch.Size([256, 12, 512])
tag_space: torch.Size([256, 12, 8855])
Epoch [3/3], Step [134/1618], Loss: 1.9342, Perplexity: 6.9182lstm_out: torch.Size([256, 12, 512])
tag_space: torch.Size([256, 12, 8855])
Epoch [3/3], Step [135/1618], Loss: 2.0174, Perplexity: 7.5190lstm_out: torch.Size([256, 9, 512])
tag_space: torch.Size([256, 9, 8855])
Epoch [3/3], Step [136/1618], Loss: 2.3500, Perplexity: 10.4856lstm_out: torch.Size([256, 14, 512])
tag_space: torch.Size([256, 14, 8855])
Epoch [3/3], Step [137/1618], Loss: 2.0775, Perplexity: 7.9843lstm_out: torch.Size([256, 12, 512])
tag_space: torch.Size([256, 12, 8855])
Epoch [3/3], Step [138/1618], Loss

Epoch [3/3], Step [190/1618], Loss: 1.9602, Perplexity: 7.1008lstm_out: torch.Size([256, 17, 512])
tag_space: torch.Size([256, 17, 8855])
Epoch [3/3], Step [191/1618], Loss: 2.3294, Perplexity: 10.2722lstm_out: torch.Size([256, 11, 512])
tag_space: torch.Size([256, 11, 8855])
Epoch [3/3], Step [192/1618], Loss: 2.0948, Perplexity: 8.1240lstm_out: torch.Size([256, 16, 512])
tag_space: torch.Size([256, 16, 8855])
Epoch [3/3], Step [193/1618], Loss: 2.1394, Perplexity: 8.4944lstm_out: torch.Size([256, 17, 512])
tag_space: torch.Size([256, 17, 8855])
Epoch [3/3], Step [194/1618], Loss: 2.3947, Perplexity: 10.9654lstm_out: torch.Size([256, 11, 512])
tag_space: torch.Size([256, 11, 8855])
Epoch [3/3], Step [195/1618], Loss: 2.1253, Perplexity: 8.3754lstm_out: torch.Size([256, 16, 512])
tag_space: torch.Size([256, 16, 8855])
Epoch [3/3], Step [196/1618], Loss: 2.2339, Perplexity: 9.3358lstm_out: torch.Size([256, 14, 512])
tag_space: torch.Size([256, 14, 8855])
Epoch [3/3], Step [197/1618], Lo

Epoch [3/3], Step [249/1618], Loss: 2.0060, Perplexity: 7.4337lstm_out: torch.Size([256, 12, 512])
tag_space: torch.Size([256, 12, 8855])
Epoch [3/3], Step [250/1618], Loss: 1.9939, Perplexity: 7.3441lstm_out: torch.Size([256, 14, 512])
tag_space: torch.Size([256, 14, 8855])
Epoch [3/3], Step [251/1618], Loss: 2.0632, Perplexity: 7.8710lstm_out: torch.Size([256, 19, 512])
tag_space: torch.Size([256, 19, 8855])
Epoch [3/3], Step [252/1618], Loss: 2.5987, Perplexity: 13.4465lstm_out: torch.Size([256, 11, 512])
tag_space: torch.Size([256, 11, 8855])
Epoch [3/3], Step [253/1618], Loss: 2.1230, Perplexity: 8.3563lstm_out: torch.Size([256, 13, 512])
tag_space: torch.Size([256, 13, 8855])
Epoch [3/3], Step [254/1618], Loss: 1.9600, Perplexity: 7.0994lstm_out: torch.Size([256, 14, 512])
tag_space: torch.Size([256, 14, 8855])
Epoch [3/3], Step [255/1618], Loss: 2.0127, Perplexity: 7.4838lstm_out: torch.Size([256, 13, 512])
tag_space: torch.Size([256, 13, 8855])
Epoch [3/3], Step [256/1618], Los

Epoch [3/3], Step [308/1618], Loss: 1.9578, Perplexity: 7.0840lstm_out: torch.Size([256, 12, 512])
tag_space: torch.Size([256, 12, 8855])
Epoch [3/3], Step [309/1618], Loss: 1.9090, Perplexity: 6.7461lstm_out: torch.Size([256, 13, 512])
tag_space: torch.Size([256, 13, 8855])
Epoch [3/3], Step [310/1618], Loss: 1.9894, Perplexity: 7.3115lstm_out: torch.Size([256, 13, 512])
tag_space: torch.Size([256, 13, 8855])
Epoch [3/3], Step [311/1618], Loss: 2.0480, Perplexity: 7.7527lstm_out: torch.Size([256, 13, 512])
tag_space: torch.Size([256, 13, 8855])
Epoch [3/3], Step [312/1618], Loss: 1.9073, Perplexity: 6.7347lstm_out: torch.Size([256, 11, 512])
tag_space: torch.Size([256, 11, 8855])
Epoch [3/3], Step [313/1618], Loss: 1.9974, Perplexity: 7.3700lstm_out: torch.Size([256, 13, 512])
tag_space: torch.Size([256, 13, 8855])
Epoch [3/3], Step [314/1618], Loss: 2.0020, Perplexity: 7.4042lstm_out: torch.Size([256, 17, 512])
tag_space: torch.Size([256, 17, 8855])
Epoch [3/3], Step [315/1618], Loss

KeyboardInterrupt: 

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.